In [13]:
# Date of creation: 12-12-24
# Author: David Yang

### **Step 1: CpG Methylation Probability Pileup QC**

CpG methylation probability pileups generated by [pb-CpG-tools](https://github.com/PacificBiosciences/pb-CpG-tools) (v2.3.2) can produce incorrect results in two key scenarios where there is a heterozygous variant:

1. **Variant Sites Appearing as Unmethylated CpGs**: When genetic variants disrupt CpG sites, these positions can be misinterpreted as CpGs with 0%/low methylation (>10%) probability. This creates "phantom" unmethylated CpG calls at variant locations. 
2. **Denovo CpG Creation**: Variants can create new CpG sites on one haplotype that don't exist on the other. This leads to incorrect methylation calls when the non-CpG haplotype is interpreted as having unmethylated CpGs.

#### Workflow

**Input Files Required:**
- VCF file containing genetic variants
- Reference FASTA file
- CpG methylation BED files (combined, hap1, hap2)

**Analysis Steps:**
1. Identify variant positions affecting CpG sites
2. Detect destroyed reference CpGs and created denovo CpGs per haplotype
3. Filter methylation calls based on variant effects
4. Generate QC metrics and visualizations

**Output Files:**
- Filtered BED files with problematic sites removed
- Excluded sites BED files
- QC report with filtering statistics
- Distribution plots for passing and excluded sites


### Filter out phantom CpG sites:

In [3]:
        # Define the base directory
base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"

# Construct paths using the base directory
bash_dir = f"{base_dir}/bash/pileup_QC"
sample_list = f"{base_dir}/bash/pileup_QC/sample_list.txt"
config_file = f"{base_dir}/bash/pileup_QC/config.yaml"
output_dir = f"{base_dir}/outputs/pileup_QC"

# Run the filtering script
!sbatch $bash_dir/submit_pileup_qc.sh $sample_list $config_file $output_dir

Submitted batch job 8765205


### Visualize the results of the filtered CpG sites.

In [13]:
# Update paths for summary plots.
import os
base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
input_dir = f"{base_dir}/outputs/pileup_QC"
output_dir = f"{base_dir}/outputs/pileup_QC/plots"
python_script = f"{base_dir}/python/pileup_QC/summarize_qc.py"

# Run the script
import subprocess
subprocess.run(["python", python_script, "--input-dir", input_dir, "--output-dir", output_dir])

2025-01-07 17:22:45,633 - INFO - Starting CpG plot generation
2025-01-07 17:22:45,914 - INFO - Loaded data from /gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/outputs/pileup_QC/SPM320/filtering_summary.csv
2025-01-07 17:22:45,920 - INFO - Loaded data from /gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/outputs/pileup_QC/SPM180/filtering_summary.csv
2025-01-07 17:22:45,926 - INFO - Loaded data from /gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/outputs/pileup_QC/SPM321/filtering_summary.csv
2025-01-07 17:22:45,933 - INFO - Loaded data from /gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/outputs/pileup_QC/SPM161/filtering_summary.csv
2025-01-07 17:22:45,939 - INFO - Loaded data from /gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/outputs/pileup_QC/SPM303/filtering_summary.csv
2025-01-07 17:22:4

CompletedProcess(args=['python', '/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/python/pileup_QC/summarize_qc.py', '--input-dir', '/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/outputs/pileup_QC', '--output-dir', '/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/outputs/pileup_QC/plots'], returncode=0)

#### Note
A site is considered "preserved" if it meets these criteria:
- Has high methylation probability (>90%)
- Has good coverage (≥10 reads)
- Was initially marked for exclusion but isn't at a variant position


### Generate plots of the CpG pileup distribution

In [26]:
import os

# Generates a plot per sample which shows the distribution of CpG methylation pileups.

base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir = f"{base_dir}/bash/pileup_QC"
input_dir = f"{base_dir}/outputs/pileup_QC"
output_dir = f"{base_dir}/outputs/pileup_QC/plots"

!sbatch $bash_dir/pileup_distribution.sh $input_dir $output_dir



Submitted batch job 8765383


### **Step 2: Generate segmentation data**


In [2]:
import os
# Run the segmentation script on QCed CpG data
base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir = f"{base_dir}/bash/segmentation"
input_dir = f"{base_dir}/outputs/pileup_QC"
output_dir = f"{base_dir}/outputs/segmentation/QC"

!sbatch $bash_dir/a_segment_samples.sh $input_dir $output_dir


Submitted batch job 8766345


In [3]:

import os
# Run the segmentation script on original CpG data

!sbatch $bash_dir/segment_original.sh



Submitted batch job 8766364


In [6]:
# Parse the segmentation results of the QCed data

base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir = f"{base_dir}/bash/segmentation"
input_dir = f"{base_dir}/outputs/segmentation/QC"
output_dir = f"{base_dir}/outputs/segmentation/QC/regions_by_label"

!sbatch $bash_dir/b_extract-regions.sh $input_dir $output_dir 




Submitted batch job 8766476


In [4]:
# Parse the segmentation results of the original data

!sbatch $bash_dir/extract-original.sh

Submitted batch job 8766398


### Segmentation Plots of QC-ed data

In [10]:
import os
# Summarize the segmentation results for each sample.
# Summary plots of the region counts and region sizes


base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir = f"{base_dir}/bash/segmentation"
input_dir = f"{base_dir}/outputs/segmentation/QC"
output_dir = f"{base_dir}/outputs/segmentation/QC"


!sbatch $bash_dir/plot_segmentation_sample.sh $input_dir $output_dir
!sbatch $bash_dir/plot_segmentation_group.sh $input_dir $output_dir


Submitted batch job 8766480
Submitted batch job 8766481


### Segmentation plots of original data


In [16]:
import os
# Summarize the segmentation results for each sample.
# Summary plots of the region counts and region sizes


base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir = f"{base_dir}/bash/segmentation/no-QC"
input_dir = f"{base_dir}/outputs/segmentation/no-QC"
output_dir = f"{base_dir}/outputs/segmentation/no-QC"


!sbatch $bash_dir/plot_segmentation_sample.sh $input_dir $output_dir
!sbatch $bash_dir/plot_segmentation_group.sh $input_dir $output_dir


Submitted batch job 8766705


Submitted batch job 8766706


#### QC VS No-QC CpG data
The script below generates a heat map comparing the segmentation region summary data (counts and sizes).
- The percentage values are the % difference between the original data and the QC data.

Observations:
SPM282 appears to have been affected by the QC the most,w ehreby the unmethylated region count decreased by ~22%, and the sizes of tehse regions increase by 20%

In [6]:
#!/usr/bin/env python3
"""
Generate heatmap visualization from delta values
Usage: python generate_heatmap.py
"""
import plotly.graph_objects as go
import pandas as pd

# Create the data
data = {
    'sample': ['SPM161', 'SPM180', 'SPM187', 'SPM198', 'SPM202', 'SPM206', 'SPM212', 
               'SPM231', 'SPM248', 'SPM276-1', 'SPM282', 'SPM296', 'SPM300', 'SPM303', 
               'SPM314', 'SPM320', 'SPM321', 'Average Delta'],
    'Hap1_M_count_delta': [-2, -2, -2, -2, -2, -2, -2, -2, 5, -10, 0, -2, 7, -6, -5, -3, -3, -2],
    'Hap1_M_mean_size_delta': [-1, -1, -1, -1, -1, -1, -1, -1, -3, -2, 0, -1, 0, 1, -6, 0, -2, -1],
    'Hap1_M_std_size_delta': [-1, -1, -1, -1, -1, -1, -1, -2, -4, -3, 1, 2, -4, 1, -3, 2, -5, -1],
    'Hap1_U_count_delta': [2, 0, 4, 3, 3, 4, 4, 4, 8, 9, -22, 17, 1, -3, 22, -7, -5, 3],
    'Hap1_U_mean_size_delta': [4, 7, 2, 3, 3, 2, 3, 2, -6, 7, 18, -6, -4, -4, -1, 14, 18, 4],
    'Hap1_U_std_size_delta': [3, 6, 2, 2, 3, 1, 3, 2, -7, 7, 18, -8, -3, -4, -3, 13, 18, 3],
    'Hap2_M_count_delta': [-2, -2, -2, -2, -2, -2, -2, -2, -2, -10, 0, -2, 7, -6, -4, -3, -3, -2],
    'Hap2_M_mean_size_delta': [-1, -1, -1, -1, -1, -1, -1, -1, -2, -3, 0, -1, 0, 1, -5, 0, -2, -1],
    'Hap2_M_std_size_delta': [-1, -1, -1, -1, -1, -1, -1, -1, -2, -3, 1, 1, -4, 0, -2, 2, -4, -1],
    'Hap2_U_count_delta': [2, -1, 4, 3, 3, 4, 4, 4, 8, 8, -21, 17, 1, -2, 22, -6, -5, 3],
    'Hap2_U_mean_size_delta': [4, 7, 2, 3, 3, 2, 3, 2, -6, 7, 16, -7, -3, -3, -1, 13, 18, 4],
    'Hap2_U_std_size_delta': [4, 6, 3, 3, 3, 2, 3, 2, -7, 10, 16, -8, -3, -3, -4, 11, 20, 3]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=df.iloc[:, 1:].values,
    x=df.columns[1:],
    y=df['sample'],
    text=df.iloc[:, 1:].values,
    texttemplate='%{text}%',
    textfont={"size": 10},
    colorscale='RdBu_r',
    zmid=0,
    colorbar_title="Delta %"
))

# Update layout
fig.update_layout(
    title='Delta Values Heatmap',
    xaxis_title="Metrics",
    yaxis_title="Sample",
    width=1200,
    height=800,
    xaxis={'tickangle': 45}
)

# Save as HTML file (interactive)
fig.write_html("heatmap.html")

# Show plot in notebook
fig.show()

### **Step 3: Generate candidate regions of interest**

These are generated by aggregating overlapping unmethylated regions across samples 

In [31]:
# Generate the consensus regions for the QC data

base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir = f"{base_dir}/bash/candidate_regions"
input_dir = f"{base_dir}/outputs/segmentation/QC/regions_by_label/regions"
output_dir = f"{base_dir}/outputs/candidate_regions/QC/pre-filter"

!sbatch $bash_dir/a_consensus_regions.sh $input_dir $output_dir

Submitted batch job 8880266


QC candidate region summary:

H1 Statistics:
Average number of regions per sample: 75022.35 ± 10555.37
Average region size across samples: 749.45 ± 95.52
Consensus regions: 205253 regions
Consensus region size: 1004.91 ± 853.61

H2 Statistics:
Average number of regions per sample: 74935.18 ± 10593.65
Average region size across samples: 748.02 ± 94.27
Consensus regions: 205402 regions
Consensus region size: 999.74 ± 843.86

In [26]:
# Generate the consensus regions for the no-QC data

base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir = f"{base_dir}/bash/candidate_regions"
input_dir = f"{base_dir}/outputs/segmentation/no-QC/regions_by_label"
output_dir = f"{base_dir}/outputs/candidate_regions/no-QC/pre-filter"

!sbatch $bash_dir/a_consensus_regions.sh $input_dir $output_dir

Submitted batch job 8880252


no-QC candidate region summary:
H1 Statistics:
Average number of regions per sample: 76923.47 ± 11355.56
Average region size across samples: 774.60 ± 90.45
Consensus regions: 228619 regions
Consensus region size: 1006.24 ± 861.71

H2 Statistics:
Average number of regions per sample: 76851.35 ± 11411.47
Average region size across samples: 773.28 ± 89.50
Consensus regions: 228659 regions
Consensus region size: 1002.55 ± 852.45


### Filter candidate regions to generate refined candidate loci based on the overlap with contiguous methylated regions.


In [34]:
# Filter QC candidate regions to generate refined candidate loci
import os

base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir = f"{base_dir}/bash/candidate_regions"
candidate_dir = f"{base_dir}/outputs/candidate_regions/QC/pre-filter"
seg_region_dir = f"{base_dir}/outputs/segmentation/QC/regions_by_label/regions"
output_dir = f"{base_dir}/outputs/candidate_regions/QC/filtered"

print(bash_dir)

!sbatch $bash_dir/b_filter-consensus.sh $candidate_dir $seg_region_dir $output_dir H1
!sbatch $bash_dir/b_filter-consensus.sh $candidate_dir $seg_region_dir $output_dir H2

/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/bash/candidate_regions
Submitted batch job 8880267
Submitted batch job 8880268


In [36]:
# Filter candidate regions to generate refined candidate loci
import os

base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir = f"{base_dir}/bash/candidate_regions"
candidate_dir = f"{base_dir}/outputs/candidate_regions/no-QC/pre-filter"
seg_region_dir = f"{base_dir}/outputs/segmentation/no-QC/regions_by_label"
output_dir = f"{base_dir}/outputs/candidate_regions/no-QC/filtered"

!sbatch $bash_dir/b_filter-consensus.sh $candidate_dir $seg_region_dir $output_dir H1
!sbatch $bash_dir/b_filter-consensus.sh $candidate_dir $seg_region_dir $output_dir H2




Submitted batch job 8880283
Submitted batch job 8880284


### **Step 4: Generate IGV viewable files**
For visualizing our candidate regions and variants, we will need to aggregate, format, and colorize the 2 datasets generated:
- Individual sample data
- Candidate regions

Files generated:

- allele_stacks :  IGV colorized bed files of each sample's segmentation regions organized by haplotype.
- candidate_regions : IGV colorized bed files of the candidate regions

In [39]:
# Generate allele stacks of the QC-ed segmentation by aggregating the segmentation data for each sample

# Set up paths
base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir = f"{base_dir}/bash/igv_viewing"
INPUT_DIR=f"{base_dir}/outputs/segmentation/QC/regions_by_label/regions"
OUTPUT_DIR=f"{base_dir}/outputs/igv_beds/QC/allele_stacks"


!sbatch $bash_dir/a_IGV_all-samples.sh $INPUT_DIR $OUTPUT_DIR


Submitted batch job 8880296


In [40]:
# Generate allele stacks of the no-QC segmentation by aggregating the segmentation data for each sample

# Set up paths
base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir = f"{base_dir}/bash/igv_viewing"
INPUT_DIR=f"{base_dir}/outputs/segmentation/no-QC/regions_by_label"
OUTPUT_DIR=f"{base_dir}/outputs/igv_beds/no-QC/allele_stacks"


!sbatch $bash_dir/a_IGV_all-samples.sh $INPUT_DIR $OUTPUT_DIR


Submitted batch job 8880298


In [ ]:
# Generate IGV beds for the QC candidate regions

# Set up paths
base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir = f"{base_dir}/bash/igv_viewing"
INPUT_DIR=f"{base_dir}/outputs/candidate_regions/QC/filtered""
OUTPUT_DIR=f"{base_dir}/outputs/igv_beds/QC/candidate_regions"


!sbatch $bash_dir/b_consensus-IGV.sh $INPUT_DIR $OUTPUT_DIR


In [ ]:
# Generate IGV beds for the no-QC candidate regions

# Set up paths
base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir = f"{base_dir}/bash/igv_viewing"
INPUT_DIR=f"{base_dir}/outputs/candidate_regions/no-QC/filtered""
OUTPUT_DIR=f"{base_dir}/outputs/igv_beds/no-QC/candidate_regions"


!sbatch $bash_dir/b_consensus-IGV.sh $INPUT_DIR $OUTPUT_DIR


In [ ]:
# Create BED files for IGV viewing each sample's H1 and H2 segments

# Define paths
input_dir="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/AlleleStacker/outputs/segmentation/regions_by_label/regions"
output_dir="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/AlleleStacker/old_QC_AS/outputs/allele_stacks/all_samples/sample_segmentation"

!sbatch $bash_current/c_IGV-segmentation.sh $input_dir $output_dir



### **Step 5: Prepare VCFs for variant mapping**
This requires merging each sample's VCF for each variant type into a cohort wide VCF.
* Adjust file paths within each bash script

Each VCF was QC'ed with the following parameters for this run :


Merging:
bcftools merge
    --force-samples \
    --merge both \
    --file-list "$FILEPATHS" \
    --output-type z \

Small variants:
bcftools filter -i 'QUAL>=20 && DP>=10' 
bcftools annotate --set-id '%CHROM:%POS:%REF:%ALT'

SVs:
bcftools filter -i 'FILTER="PASS" && DP>=10'
bcftools annotate --set-id '%CHROM:%POS:%REF:%ALT:%SVANN'

CNVs:
bcftools filter -i 'QUAL>=20 && FILTER="PASS"
bcftools annotate --set-id '%CHROM:%POS:%REF:%ALT:%SVLEN' 

Tandem Repeats:
bcftools filter -i 'QUAL>=20 && FILTER="PASS"'
bcftools annotate --set-id '%CHROM:%POS:%REF:%ALT'

In [6]:

# Set up environment
import os
import sys

# Set up paths
bash_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/bash"
bash_current = os.path.join(bash_dir, "variant_map")   

In [7]:
# merge each VCF type into a single cohort vcf using bcftools merge
# small variants
FILEPATHS="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/bash/variant_map/file_paths/filepaths_small.txt"
OUT_DIR="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/outputs/variant_mapping/merged_variants/small_variants"


!sbatch $bash_current/a_merge_small_vars.sh $FILEPATHS $OUT_DIR

Submitted batch job 8765225


In [8]:
# merge structural variants

FILEPATHS="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/bash/variant_map/file_paths/filepaths_SVs.txt"
OUT_DIR="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/outputs/variant_mapping/merged_variants/structural_variants"


!sbatch $bash_current/b_merge_sv_vars.sh $FILEPATHS $OUT_DIR

Submitted batch job 8765226


In [9]:
# merge copy number variants
FILEPATHS="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/bash/variant_map/file_paths/filepaths_CNVs.txt"
OUT_DIR="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/outputs/variant_mapping/merged_variants/copy_number_variants"

!sbatch $bash_current/c_merge_CNVs_vars.sh $FILEPATHS $OUT_DIR

Submitted batch job 8765227


In [10]:
# merge tandem repeats

FILEPATHS="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/bash/variant_map/file_paths/filepaths_repeats.txt"

OUT_DIR="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker/outputs/variant_mapping/merged_variants/trgt_repeat_vcf"

!sbatch $bash_current/d_merge_repeats_vars.sh  $FILEPATHS $OUT_DIR

# none remained after QC, omitted for now  

Submitted batch job 8765228


### Step 6: Map variants to regions

#### Haplotype-Specific Variant Mapping Analysis

Overview
Maps genetic variants to haplotype-specific methylated/unmethylated regions. Processes phased variants (SNPs, SVs) and unphased variants (CNVs, tandem repeats) from VCF files against methylation BED files.

Input Requirements
- BED file with methylation regions containing:
  - Chromosome, start, end positions
  - `methylated_samples` and `unmethylated_samples` columns
- Indexed VCF files (.vcf.gz + .tbi) for variants:
  - Small variants (SNPs/indels)
  - Copy number variants
  - Structural variants  
  - Tandem repeats

Output Format
Tab-separated file with columns:
```
chrom         Chromosome
start         Region start position
end           Region end position
variant_id    Unique variant identifier 
type          Variant type (small/cnv/sv/tr)
ref           Reference allele
alt           Alternative allele
num_meth      Number of samples with variant + methylated
num_unmeth    Number of samples with variant + unmethylated
meth_samples  Methylated samples with variant: sample:genotype
unmeth_samples Unmethylated samples with variant: sample:genotype
```
Haplotype Assignment
- Phased variants (small, sv): Uses `|`-separated genotypes, checking H1 (idx 0) or H2 (idx 1)
- Unphased variants (cnv, tr): Reports any non-reference genotype



In [4]:
# Set up environment
import os
import sys

# Set up paths
base_dir = "/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/QC_Comparison/AlleleStacker"
bash_dir ="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/1-14-25/AlleleStacker/bash/variant_map"
BED_DIR=f"{base_dir}/outputs/candidate_regions/QC/filtered"
VARIANT_DIR=f"{base_dir}/outputs/variant_mapping/merged_variants"
PYTHON_DIR="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/1-14-25/AlleleStacker/python/variant_map"
OUTPUT_DIR="/gs/gsfs0/shared-lab/greally-lab/David/AlleleStacker_tests/1-14-25/AlleleStacker/outputs/variant_mapping/results"  

# SUBMIT THE JOB
!sbatch $bash_dir/e_map_variants.sh $BED_DIR $VARIANT_DIR $PYTHON_DIR $OUTPUT_DIR

Submitted batch job 8885144
